In [24]:
from dask.distributed import Client
import dask.dataframe as dd
import pandas as pd
import time
import statistics

import xgboost as xgb
import dask.array as da
import dask.distributed

In [25]:
times=[]

In [26]:
client = Client('172.31.45.128:8786')
client.restart()
print(client)

<Client: 'tcp://172.31.45.128:8786' processes=0 threads=0, memory=0 B>


In [27]:
data = dd.read_csv("/home/ubuntu/data.csv", sample=2**20)
data = data.persist()

In [28]:
train, test = data.random_split([0.7, 0.3], random_state=42)

In [29]:
X_train = train.drop(labels=["tip_amount"], axis=1).copy()
Y_train = train["tip_amount"].copy()

In [30]:
X_test = test.drop(labels=["tip_amount"], axis=1).copy()
Y_test = test["tip_amount"].copy()

# 10 Workers

In [8]:
for i in range(5):
    t0 = time.time()  # Start time

    # Create DaskDMatrix object
    dtrain = xgb.dask.DaskDMatrix(client, X_train, Y_train)

    # Train the model
    output = xgb.dask.train(client,
                            {"verbosity": 2,
                             "tree_method": "hist",
                             "objective": "reg:squarederror"},
                            dtrain,
                            num_boost_round=100,
                            evals=[(dtrain, "train")])

    t1 = time.time()  # End time

    # Calculate the duration and append to times list
    times.append(t1 - t0)

# Print out the times for each run
print("Training times for each run:", times)

Training times for each run: [45.079017162323, 36.98415732383728, 37.29812169075012, 38.90826678276062, 38.73184633255005]


In [9]:
times=[]

In [10]:
for i in range(5):
    t0 = time.time()
    prediction = xgb.dask.predict(client, output['booster'], X_test)
    t1 = time.time()
    times.append(t1-t0)
print("Times for each run:", times)

Times for each run: [0.12967967987060547, 0.050623178482055664, 0.03557467460632324, 0.03759455680847168, 0.10413694381713867]


In [11]:
import dask.array as da

# Assuming 'prediction' and 'Y_test' are defined properly
Y_test_array = Y_test.to_dask_array(lengths=True)

# Calculate Mean Squared Error
mse = da.mean((prediction - Y_test_array) ** 2)
rmse = da.sqrt(mse)

# Calculate Mean Absolute Error
mae = da.mean(da.absolute(prediction - Y_test_array))

# Calculate R^2 Score
ss_res = da.sum((prediction - Y_test_array) ** 2)
ss_tot = da.sum((Y_test_array - da.mean(Y_test_array)) ** 2)
r2_score = 1 - (ss_res / ss_tot)

# Compute the results
mse_result = mse.compute()
rmse_result = rmse.compute()
mae_result = mae.compute()
r2_score_result = r2_score.compute()

print(f"Mean Squared Error: {mse_result}")
print(f"Root Mean Squared Error: {rmse_result}")
print(f"Mean Absolute Error: {mae_result}")

Mean Squared Error: 0.06752249726926332
Root Mean Squared Error: 0.25985091354325335
Mean Absolute Error: 0.045129086410789686


In [12]:
times

[0.12967967987060547,
 0.050623178482055664,
 0.03557467460632324,
 0.03759455680847168,
 0.10413694381713867]

# 6 Workers

In [13]:
times=[]

In [14]:
for i in range(5):
    t0 = time.time()  # Start time

    # Create DaskDMatrix object
    dtrain = xgb.dask.DaskDMatrix(client, X_train, Y_train)

    # Train the model
    output = xgb.dask.train(client,
                            {"verbosity": 2,
                             "tree_method": "hist",
                             "objective": "reg:squarederror"},
                            dtrain,
                            num_boost_round=100,
                            evals=[(dtrain, "train")])

    t1 = time.time()  # End time

    # Calculate the duration and append to times list
    times.append(t1 - t0)

# Print out the times for each run
print("Training times for each run:", times)

Training times for each run: [53.45803737640381, 52.990593671798706, 51.201730728149414, 50.17712068557739, 50.0114471912384]


In [15]:
times=[]

In [16]:
for i in range(5):
    t0 = time.time()
    prediction = xgb.dask.predict(client, output['booster'], X_test)
    t1 = time.time()
    times.append(t1-t0)
print("Times for each run:", times)

Times for each run: [0.08875417709350586, 0.035497188568115234, 0.03751707077026367, 0.033303260803222656, 0.046210527420043945]


In [17]:
Y_test_array = Y_test.to_dask_array(lengths=True)

# Calculate Mean Squared Error
mse = da.mean((prediction - Y_test_array) ** 2)
rmse = da.sqrt(mse)

# Calculate Mean Absolute Error
mae = da.mean(da.absolute(prediction - Y_test_array))

# Calculate R^2 Score
ss_res = da.sum((Y_test_array - prediction) ** 2)
ss_tot = da.sum((Y_test_array - da.mean(Y_test_array)) ** 2)
r2_score = 1 - (ss_res / ss_tot)

# Compute the results
mse_result = mse.compute()
rmse_result = rmse.compute()
mae_result = mae.compute()
r2_score_result = r2_score.compute()

print(f"Mean Squared Error: {mse_result}")
print(f"Root Mean Squared Error: {rmse_result}")
print(f"Mean Absolute Error: {mae_result}")

Mean Squared Error: 0.07839371739342225
Root Mean Squared Error: 0.27998878083491535
Mean Absolute Error: 0.044996633338009434


# 2 Workers

In [31]:
times=[]

In [34]:
for i in range(4):
    t0 = time.time()  # Start time

    # Create DaskDMatrix object
    dtrain = xgb.dask.DaskDMatrix(client, X_train, Y_train)

    # Train the model
    output = xgb.dask.train(client,
                            {"verbosity": 2,
                             "tree_method": "hist",
                             "objective": "reg:squarederror"},
                            dtrain,
                            num_boost_round=100,
                            evals=[(dtrain, "train")])

    t1 = time.time()  # End time

    # Calculate the duration and append to times list
    times.append(t1 - t0)

# Print out the times for each run
print("Training times for each run:", times)

Training times for each run: [130.023770570755, 123.57611346244812, 124.13330554962158, 116.86070489883423]


In [35]:
times=[]

In [36]:
for i in range(5):
    t0 = time.time()
    prediction = xgb.dask.predict(client, output['booster'], X_test)
    t1 = time.time()
    times.append(t1-t0)
print("Times for each run:", times)

Times for each run: [0.06583309173583984, 0.030716419219970703, 0.03487348556518555, 0.03010106086730957, 0.031087398529052734]


In [37]:
Y_test_array = Y_test.to_dask_array(lengths=True)

# Calculate Mean Squared Error
mse = da.mean((prediction - Y_test_array) ** 2)
rmse = da.sqrt(mse)

# Calculate Mean Absolute Error
mae = da.mean(da.absolute(prediction - Y_test_array))

# Calculate R^2 Score
ss_res = da.sum((Y_test_array - prediction) ** 2)
ss_tot = da.sum((Y_test_array - da.mean(Y_test_array)) ** 2)
r2_score = 1 - (ss_res / ss_tot)

# Compute the results
mse_result = mse.compute()
rmse_result = rmse.compute()
mae_result = mae.compute()
r2_score_result = r2_score.compute()

print(f"Mean Squared Error: {mse_result}")
print(f"Root Mean Squared Error: {rmse_result}")
print(f"Mean Absolute Error: {mae_result}")

Mean Squared Error: 0.029384259835020448
Root Mean Squared Error: 0.17141837659661943
Mean Absolute Error: 0.04490424135963139
